## Implementation of all baselines

In [43]:
#import of various libraries
import numpy as np
from numba import njit
from scipy.fft import fft as fft
from scipy.fft import ifft as ifft
from scipy.optimize import newton
from sklearn.preprocessing import normalize

### 1. Count Sketch Signed Random Projection (CSSRP)

In [70]:
def CSSRP(data, reduced_dim):
    
    """This method uses the idea to implemnet Count-Sketch as matrix projection. 
        Projection matxix have only one element from {+1,-1} in each column.
    
    Input description:- 
          data: real world or synthetic dataset.
          reduced_dimension: compressed dimension of the original data point
                  
         Output Description: Binary matrix of compressed dimension of a dataset..
         
      The output from CSSRP can then be used with method Angle_estimation for pairwise angle estimation.
      
      Note: Method is not optimized for fast sketch time. 
    """
    # Variable used to generate projection matrix
    temp = np.zeros((reduced_dim,data.shape[1]))
    
    # Method to create matrix having only one non zero element from {+1,-1} in each column.
    values = np.random.choice([-1,1],data.shape[1])
    y = np.arange(0,data.shape[1])
    x = np.random.randint(0,temp.shape[0],temp.shape[1])
    temp[x,y]=values 
    
    # Compressed the dataset.
    temp = np.dot(temp,data.T)
    
    # Assign sign to every element.
    temp[temp>=0]=1
    temp[temp<0]=0
    
    return temp.T

### 2. Count Sketch Signed Random Projection-L (CSSRP-L)

In [71]:
def CSSRPL(data, l, reduced_dim):
    
    """This method uses the idea to generate projection matxix have l element 
         from {+1,-1} in each column.
    
    Input description:- 
          data: real world or synthetic dataset.
          l: no of elemnts per column. 
          reduced_dimension: compressed dimension of the original data point
                  
         Output Description: Binary matrix of compressed dimension of a dataset..
         
      The output from CSSRP-L can then be used with method Angle_estimation for pairwise angle estimation.
      
      Note: Method is not optimized for fast sketch time. 
    """
    
    # Variable used to generate projection matrix
    temp = np.zeros((reduced_dim,data.shape[1]))
    
    # Generate l no of non-zero element for each column from set {+1,-1}
    col_val = np.arange(temp.shape[0])
    for i in range(temp.shape[1]):
        np.random.shuffle(col_val)
        g = np.random.choice([-1,1],l)
        temp[col_val[:l],i] = g
    
    # Compressed the dataset.    
    temp = np.dot(temp,data.T)
    
    # Assign sign to every element.
    temp[temp>=0]=1
    temp[temp<0]=0
    
    return temp.T

### 3. Signed Random Projection (SRP)

In [72]:
def SRP(Data, R):
    
    """ This method is an implementation of Simhash (SRP) proposed in the paper- "Similarity estimation 
    techniques from rounding algorithms" the url for the paper- https://dl.acm.org/doi/pdf/10.1145/509907.509965 
        
    SRP uses Locality Sensitive Hashing Techinque, for estimatingthe angular similarity, achived by taking a 
    random projection matrix sampled form N(0,1) of (d,k) dimensions where d, k is the original dimension and 
    compressed dimension respectively, and multiplication with original dataset and take sign of the evey element.
        
    Input Description:
        Data: Orignal Dataset 
           R: Random projection matrix.
           
    Output Description: Binary matrix of compressed dimension of a dataset.
    
    """
    # Compress the dataset
    temp = np.dot(Data,R)
    
    # Assign sign to each element
    temp[temp >= 0] = 1
    temp[temp < 0] = 0
    
    return temp


In [73]:
def Angle_estimation(Data, pairs, reduced_dimension):
    
    """This method estimate the angular similarity between all the pairs of points in the compresed dataset, 
       Humming distance method has been used to estimate the angular similarity: the method is proposed 
       in a paper:- https://dl.acm.org/doi/pdf/10.1145/509907.509965
       
       Input description:- 
           Data: Compressed Binary dataset matrix.
           Piars: all possible pairs between the pairs.
           reduced_dimension: compressed dimnesion of the original data point
        
         Output Description: All the estimated angular similarity between all the pairs in a dataset.
       
       """
    
    # Variable to store the angular similarity between all the pairs of dataset.
    temp = np.zeros(pairs.shape[0])
    
    # First calculating the humming distance between each pair of dataset and then estimating the angular similarity.
    for i in range(pairs.shape[0]):
        ham = np.count_nonzero(Data[pairs[i,0]]!=Data[pairs[i,1]]) 
        temp[i] = (np.pi/reduced_dimension)*ham
        
    return temp


### 4. Circulant Binary Embedding (CBE)

In [74]:
def CBE(data, reduced_dimension):
    
    """This method is the implementaion of the work proposed in the paper Circulant Binary Embedding(CBE), the url 
    for the paper- htps://arxiv.org/pdf/1405.3162.pdf
    
    CBE uses the a matrix called circulant binary matrix of d dimnesion sampled from N(0,1) and the efficient
    use of Fast Fourier Transform.
    
    Input description:- 
           Data: Compressed Binary dataset matrix.
           reduced_dimension: compressed dimension of the original data point
        
         Output Description: Binary matrix of compressed dimension of a dataset..
    
    The output from CBE can then be used with method Angle_estimation for pairwise angle estimation.
    
    """
    
    # Sampling vector from normal distribution
    w = np.random.normal(0,1,data.shape[1]) 
    
    # Generate a vector containing elements from {+1,-1}
    D = np.random.choice([1, -1],data.shape[1])
    data = np.multiply(D,data)
    
    # Use of fft 
    temp = ifft(np.multiply(fft(w),fft(data)))[:,:int(reduced_dimension)]
    
    # Assign sign to every element.
    temp[temp>=0]=1
    temp[temp<0]=0
    
    return temp


### 5. Super-Bit

In [75]:
def Super_Bit(data, R, reduced_dimension):
    
    """This method is the implemantaion of work proposed in the paper Super-Bit Locality-Sensitive Hashing
    (Super-Bit), the url for the paper- 
                https://proceedings.neurips.cc/paper/2012/file/072b030ba126b2f4b2374f342be9ed44-Paper.pdf
    
    Super-Bit use the Gram–Schmidt process to orthoganalize the random projection matrix used by SRP
    
    Input description:- 
          data: real world or synthetic dataset.
          R: random projection matrix used for orthoganalization.
          reduced_dimension: compressed dimension of the original data point
                  
         Output Description: Binary matrix of compressed dimension of a dataset.
         
      The output from Supe_Bit can then be used with method Angle_estimation for pairwise angle estimation.
      
    """
    # Variable to store ortogonize vectors.
    W = np.zeros(R.shape)
    
    # Normalize random rojection matrix
    H = normalize(R,axis=0)
    
    # Implementation of Gram-Schmit process:
    for j in range(1,reduced_dimension) :
        W[:,j] = H[:,j]
        for k in range(j):
            W[:,j] = W[:,j]-np.dot(W[:,k],W[:,j])*H[:,j]
        W[:,j] = W[:,j]/np.linalg.norm(W[:,j])
    
    # Compressed the dataset.
    temp = np.dot(data,W)
    
    # Assign sign to every element.
    temp[temp>=0]=1
    temp[temp<0]=0
    
    return temp


### 6. Maximum Likelihood Estimation

#### 6.1 Singular value decomposition (SVD)

In [76]:
def SVD(data):
    
    """This method returns the angle with the extra vector used in MLE for pairwise angle estimation, 
    where vector is first singular vector from SVD estiation from the dataset.
    
    Input Description:
        data: Real-world and synthetic dataset
        
    Output: returns extra vector e and the angle between extar vector and all points in a dataset.
    
    """
    # Varaiable to store the angle.
    temp =np.zeros(data.shape[0])
    
    # Normalize the dataset.
    data = normalize(data,axis=1)
    
    # Extra vector  for MLE.
    x,y,z = linalg.svd(data)
    t = np.argmax(y)
    e = z[t]
    
    #angle beteen extra vetor and all the points of dataset
    for i in range(data.shape[0]):
        temp[i] = np.arccos(np.dot(data[i],e))

    return temp, e


#### 6.2  Polynomial Function For MLE

In [77]:
def Quad_function(p3, theta_x1_e, theta_x2_e, n1, n2, n3, n4):
    
    """Implementation of the equation for pairwise angular estimation used by MLE"""
    
    temp1 =  n1*np.pi*(theta_x1_e + np.pi*(p3-1))*(theta_x1_e + theta_x2_e + np.pi*(p3-1))*p3
    temp2 =  n4*np.pi*(theta_x2_e + np.pi*(p3-1))*(theta_x1_e + theta_x2_e + np.pi*(p3-1))*p3
    temp3 =  n2*np.pi*(theta_x2_e + np.pi*(p3-1))*(theta_x1_e + np.pi*(p3-1))*p3
    temp4 =  n3*(theta_x1_e + np.pi*(p3-1))*(theta_x2_e + np.pi*(p3-1))*(theta_x1_e + theta_x2_e + np.pi*(p3-1))
    
    
    return temp1+temp2+temp3+temp4


#### 6.3 Angle estimation via MLE

In [78]:
def MLE(V, pairs, ve, angle_e, K):
    
    """This method is the implemantaion of work proposed in the paper Improving Sign Random Projections
    With Additional Information(CBE), the url for the paper-http://proceedings.mlr.press/v80/kang18b/kang18b.pdf
    
    MLE uses the an extra vector e along with the method called Maximum likelihood estimation 
    to estimate pair wise angular similarity.
    
    Input description:- 
          V: Binary vector derived from SRP method.
          pairs: all possible pairs of data vectors of the dataset.
          ve: reduced vector computed from random projection matrix used by SRP multiplied by extra vector e 
                  computed from SVD, and taking sign of the vector.
                  
         Output Description: All the estimated angular similarity between all the pairs in a dataset.
    
    """
    
    # Variable to store pairwise angle between points of dataset
    temp = np.zeros(pairs.shape[0])
    
    # Implementation of the estimator 
    for i in range(pairs.shape[0]):
        n1=n2=n3=n4=0
        X = np.vstack((V[pairs[i,0]]*4, V[pairs[i,1]]*2, e))
        X = np.sum(X,axis=0)
        n1= len(X[X==3])+len(X[X==4])
        n2= len(X[X==1])+len(X[X==6])
        n3= len(X[X==7])+len(X[X==0])
        n4= len(X[X==5])+len(X[X==2])
        n = n1+n2+n3+n4
        cc = n3/n
        try : 
            # Use of newton rapson 
            p3 = newton(Quad_function,cc,args=(angle_e[pairs[i,0]],angle_e[pairs[i,1]],n1,n2,n3,n4),maxiter=1000)
        except:
            print("NOT CONVERGED")
            p3 = cc
            
        p2 = p3-1+((angle_e[pairs[i,0]]+angle_e[pairs[i,1]])/np.pi)
        
        # pairwise angle estimation
        temp[i] = (np.pi*(1-p2-p3))
        
    return temp
